In [1]:
import pandas as pd
from helpers import config
import plotly.express as px

# Loading results

In [75]:
best_lags = pd.read_csv(config["files"]["results"]["all_best_lags"])

# Plotting lags

In [47]:
def rolling_avg(x,window):
    return x.rolling(5, min_periods=1).mean()

In [72]:
def get_pair_lags(df,m1,m2):
    return df[(df.market1==m1) * (df.market2==m2)]

def plot_pair_lags(df,m1,m2):
    fig = px.line(get_pair_lags(best_lags,m1,m2), x="date", y="lag", title=f"lags vs date between {m1}/{m2}")
    fig.show()

rolling_window = 20 # days
def plot_all_pairs(df):
    to_plot = df.copy()
    to_plot["pair"] = to_plot["market1"]+"_"+to_plot["market2"]
    to_keep = to_plot[["lag","pair"]].groupby("pair").mean()>0#.set_index("pair")
    to_keep = to_keep[to_keep.lag].rename(columns={"lag":"lag+"})
    to_plot = to_plot.set_index("pair").join(to_keep,how="inner").reset_index()
    # filter outliers
    quantile_95 = to_plot["lag"].abs().quantile(q=0.95)
    to_plot = to_plot[to_plot.lag.abs() <= quantile_95]
    
    # rolling average
    to_plot["lag"] = rolling_avg(to_plot.lag,rolling_window)
    
    fig = px.line(to_plot, x="date", y="lag",color="pair", title=f"lags vs date (all pairs)",log_y=False)
    fig.show()

In [76]:
plot_all_pairs(best_lags)

In [73]:
plot_all_pairs(best_lags)

In [59]:

plot_all_pairs(best_lags)

# Geographical relations

### loading geographical informations

In [ ]:

exchanges_country_suffixes_df = pd.read_csv(config["files"]["exchanges_country_suffixes"])
exchanges_country_suffixes_df["Exchange ID"] = exchanges_country_suffixes_df["Exchange ID"].apply(lambda x : str(x).replace(".",""))

exchanges_country_suffixes_dict = exchanges_country_suffixes_df[["Exchange ID","Exchange Name"]].set_index("Exchange ID").to_dict()['Exchange Name']

In [ ]:
regions_vs_mean_lag = best_lags[['market1', 'market2', 'lag']].groupby(['market1', 'market2']).mean().reset_index()

In [ ]:
regions_vs_mean_lag["region1"] = regions_vs_mean_lag["market1"].apply(lambda x : exchanges_country_suffixes_dict[x]) 
regions_vs_mean_lag["region2"] = regions_vs_mean_lag["market2"].apply(lambda x : exchanges_country_suffixes_dict[x]) 

In [ ]:
fig = px.line(to_plot[::2], x="date", y="lag",color="pair", title=f"lags vs date (all pairs)")
